## Assignment :- Data Cleaning (Pandas)

It happens all the time: someone gives you data containing malformed strings, Python, lists and missing data. How do you tidy it up so you can get on with the analysis?

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'From_To': ['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm',  
'Budapest_PaRis', 'Brussels_londOn'],  
'FlightNumber': [10045, np.nan, 10065, np.nan, 10085],  
'RecentDelays': [[23, 47], [], [24, 43, 87], [13], [67, 32]], 
'Airline': ['KLM(!)', '<Air France> (12)', '(British Airways. )', 
'12. Air France', '"Swiss Air"']})
df

,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045.0,"[23, 47]",KLM(!)
1,MAdrid_miLAN,NaN,[],<Air France> (12)
2,londON_StockhOlm,10065.0,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,NaN,[13],12. Air France
4,Brussels_londOn,10085.0,"[67, 32]","""Swiss Air"""


## Problem Statement 1

Some values in the the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so 10055 and 10075 need to be put in place. Fill in these missing numbers and make the column an integer column (instead of a float column)

In [4]:
# Filling the missing values
df.loc[1,'FlightNumber'] = 10055
df.loc[3,'FlightNumber'] = 10075

In [5]:
# converting float into int
df['FlightNumber'] = df['FlightNumber'].apply(lambda x :int(x))

In [8]:
print("DataFrame after filling the missing values and converting 'float' into 'int'")
df

DataFrame after filling the missing values and converting 'float' into 'int'


,From_To,FlightNumber,RecentDelays,Airline
0,LoNDon_paris,10045,"[23, 47]",KLM(!)
1,MAdrid_miLAN,10055,[],<Air France> (12)
2,londON_StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest_PaRis,10075,[13],12. Air France
4,Brussels_londOn,10085,"[67, 32]","""Swiss Air"""


## Problem Statement 2

The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [6]:
# To get the string befor the ' _ '
From = df['From_To'].apply(lambda x : (x.split('_'))[0])
From

0      LoNDon
1      MAdrid
2      londON
3    Budapest
4    Brussels
Name: From_To, dtype: object

In [7]:
# To get the string after the ' _ '
To = df['From_To'].apply(lambda x : (x.split('_'))[1])
To

0        paris
1        miLAN
2    StockhOlm
3        PaRis
4       londOn
Name: From_To, dtype: object

In [8]:
tempDF = df.drop(['From_To'],axis = 1) # Creating a temporary DataFrame and removing the From_To Col.
tempDF.insert(0,'From',From)
tempDF.insert(1,'To',To)
tempDF

,From,To,FlightNumber,RecentDelays,Airline
0,LoNDon,paris,10045,"[23, 47]",KLM(!)
1,MAdrid,miLAN,10055,[],<Air France> (12)
2,londON,StockhOlm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,PaRis,10075,[13],12. Air France
4,Brussels,londOn,10085,"[67, 32]","""Swiss Air"""


## Problem Statement 3

Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the strings so that only the first letter is uppercase (e.g. "londON" should become "London".) 

In [9]:
tempDF['From'] = tempDF['From'].apply(lambda x : x.capitalize())
tempDF['To'] = tempDF['To'].apply(lambda x : x.capitalize())
tempDF

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


## Problem Statement 4

Delete the From_To column from df and attach the temporary DataFrame from the previous questions. 

In [19]:
df.drop(['From_To'],axis = 1,inplace = True) # Deleting the From_to column

In [25]:
df['From'] = tempDF['From'] # adding From column from temporary DataFrame 
df['To'] = tempDF['To'] # adding To column from temporary DataFrame 
col = ['From', 'To','FlightNumber', 'RecentDelays', 'Airline']
df = df.reindex(columns = col)
df

,From,To,FlightNumber,RecentDelays,Airline
0,London,Paris,10045,"[23, 47]",KLM(!)
1,Madrid,Milan,10055,[],<Air France> (12)
2,London,Stockholm,10065,"[24, 43, 87]",(British Airways. )
3,Budapest,Paris,10075,[13],12. Air France
4,Brussels,London,10085,"[67, 32]","""Swiss Air"""


## Problem Statement 5

 In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value should be NaN. 

Expand the Series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the unwanted RecentDelays column in df with delays. 

In [30]:
# Expending cells having the list 
delays = df['RecentDelays'].apply(pd.Series)
delays = delays.rename(columns = lambda x :'Delay_' + str(x))
delays

,Delay_0,Delay_1,Delay_2
0,23.0,47.0,NaN
1,NaN,NaN,NaN
2,24.0,43.0,87.0
3,13.0,NaN,NaN
4,67.0,32.0,NaN


In [34]:
# Droping the 'RecentDelays' column
df.drop(['RecentDelays'],inplace = True,axis = 1)

In [36]:
# Adding the extracted columns
pd.concat([df[:], delays[:]], axis=1)

,From,To,FlightNumber,Airline,Delay_0,Delay_1,Delay_2
0,London,Paris,10045,KLM(!),23.0,47.0,NaN
1,Madrid,Milan,10055,<Air France> (12),NaN,NaN,NaN
2,London,Stockholm,10065,(British Airways. ),24.0,43.0,87.0
3,Budapest,Paris,10075,12. Air France,13.0,NaN,NaN
4,Brussels,London,10085,"""Swiss Air""",67.0,32.0,NaN
